# 01. Imports and Configuration
**What:** Import libraries; set seeds, paths, hyperparameters, and fixed splits.  
**Why:** Deterministic training/evaluation and reproducible packaging.  
**Method choices:** Fixed splits (Train 2021-02-03→2022-12-30; Val 2023-01-03→2023-05-31; Test 2023-06-01→2023-12-28, n=146); Target=Close.shift(-1); America/New_York 16:00 cut-off; ES on Validation; deep models use **train-only scaling** with **zero-preservation** for Tw_/Rd_/Nw_SP500_*; cadence = **monthly refit**.


In [ ]:
import os, sys, json, math, time, hashlib, platform, warnings, random, zipfile
from pathlib import Path
from typing import List, Dict, Tuple, Optional

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# -------------------- config --------------------
warnings.filterwarnings("ignore")

MODEL_ID = "LSTM_SE"
TICKERS = ["AAPL","AMZN","MSFT","TSLA","AMD"]
DATA_DIR = Path("final_inputs")
OUT_ROOT = Path("LSTM_SE_FINAL")
MODEL_DIR = OUT_ROOT / MODEL_ID
OUT_ROOT.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

TRAIN_START = "2021-02-03"
TRAIN_END   = "2022-12-30"
VAL_START   = "2023-01-03"
VAL_END     = "2023-05-31"
TEST_START  = "2023-06-01"
TEST_END    = "2023-12-28"
ASSERT_N_TEST = 146

# Reproducibility
RANDOM_SEED = 1337
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
try:
    torch.use_deterministic_algorithms(True)
except Exception:
    pass

# Modelling hyperparameters
L = 90
BATCH_SIZE = 64
EPOCHS = 150
EARLY_STOP_PATIENCE = 15
LR = 1e-3
HIDDEN = 64
LAYERS = 2
DROPOUT = 0.2
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Metrics thresholds
EPSILON_RET = 0.0010  # for DA epsilon-gated classification

# External provenance (for code adaptation disclosure)
COMMIT_SHA = "f7e54711205ca29a06577a04047b4a40df32fdec"

# Audit evidence: market clock and target provenance
CLOCK_INVARIANTS = {
    "market_timezone": "America/New_York",
    "cutoff_local_time": "16:00:00",
    "no_forward_fill_past_cutoff": True,
    "sentiment_zero_encoding_on_no_activity_days": True
}
TARGET_PROVENANCE = {
    "definition": "Target = Close.shift(-1)",
    "created_after_all_features": True
}

# Target scaling policy: standardise y on Train-only for numerically stable training;
# ALWAYS inverse-transform to level BEFORE metrics and writing predictions.
SCALE_TARGET = True

# Data Loading
**What:** Read per-ticker panels and split into Train/Val/Test by fixed dates.  
**Why:** Ensure leakage-safe, consistent windows and schema.  
**Method choices:** Dates sorted; Target column required; splits = exact dates above.


In [ ]:
# -------------------- IO helpers --------------------
def load_frame(ticker: str) -> pd.DataFrame:
    p = DATA_DIR / f"{ticker}_input.csv"
    if not p.exists():
        raise FileNotFoundError(f"Missing input file: {p}")
    df = pd.read_csv(p)
    if "date" not in df.columns or "Target" not in df.columns:
        raise RuntimeError(f"{p} must include 'date' and 'Target' columns")
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date").reset_index(drop=True)
    return df

def fixed_splits(df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    tr = df[(df["date"]>=TRAIN_START) & (df["date"]<=TRAIN_END)].copy()
    va = df[(df["date"]>=VAL_START) & (df["date"]<=VAL_END)].copy()
    te = df[(df["date"]>=TEST_START) & (df["date"]<=TEST_END)].copy()
    return {"train": tr, "val": va, "test": te}

def infer_features(df: pd.DataFrame) -> List[str]:
    drop_cols = {"date","ticker","Target"}
    return [c for c in df.columns if c not in drop_cols]

def sentiment_cols_from(features: List[str]) -> List[str]:
    return [c for c in features if c.startswith(("Tw_","Rd_","Nw_SP500_"))]

# Preprocessing
**What:** Train-only scaling: zero-preserving for sentiment; standard z-scaling for other features; optional target standardisation.  
**Why:** Stabilise optimisation while preserving zeros for Tw_/Rd_/Nw_SP500_* and avoiding look-ahead.  
**Method choices:** Fit scalers on **Train** only; apply to Val/Test; inverse-transform **y** to level before metrics/outputs.

In [ ]:
# -------------------- scalers --------------------
class ZeroPreservingStandardScaler:
    def __init__(self, columns: List[str]):
        self.columns = list(columns)
        self.mu_: Dict[str, float] = {}
        self.sd_: Dict[str, float] = {}
        self.fitted_ = False
    def fit(self, df_train: pd.DataFrame):
        for c in self.columns:
            x = df_train[c].to_numpy()
            nz = x != 0
            if nz.any():
                mu = x[nz].mean()
                sd = x[nz].std(ddof=0)
                if sd <= 0: sd = 1.0
            else:
                mu, sd = 0.0, 1.0
            self.mu_[c] = float(mu)
            self.sd_[c] = float(sd)
        self.fitted_ = True
        return self
    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        assert self.fitted_
        z = df.copy()
        for c in self.columns:
            x = z[c].to_numpy(dtype=float)
            nz = x != 0
            y = x.copy()
            y[nz] = (x[nz] - self.mu_[c]) / self.sd_[c]
            z[c] = y
        return z

class StdScaler:
    def __init__(self, columns: List[str]):
        self.columns = list(columns)
        self.mu_: Dict[str, float] = {}
        self.sd_: Dict[str, float] = {}
        self.fitted_ = False
    def fit(self, df_train: pd.DataFrame):
        for c in self.columns:
            x = df_train[c].to_numpy(dtype=float)
            mu = float(x.mean())
            sd = float(x.std(ddof=0))
            if sd <= 0: sd = 1.0
            self.mu_[c] = mu
            self.sd_[c] = sd
        self.fitted_ = True
        return self
    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        assert self.fitted_
        z = df.copy()
        for c in self.columns:
            x = z[c].to_numpy(dtype=float)
            z[c] = (x - self.mu_[c]) / self.sd_[c]
        return z

class Identity1D:
    def fit(self, y: np.ndarray): return self
    def transform(self, y: np.ndarray) -> np.ndarray: return y.astype(np.float32)
    def inverse_transform(self, y: np.ndarray) -> np.ndarray: return y.astype(np.float32)

class StdScaler1D:
    def __init__(self): self.mu = 0.0; self.sd = 1.0
    def fit(self, y: np.ndarray):
        y = np.asarray(y, dtype=np.float64)
        self.mu = float(np.mean(y))
        sd = float(np.std(y, ddof=0))
        self.sd = sd if sd > 0 else 1.0
        return self
    def transform(self, y: np.ndarray) -> np.ndarray:
        return ((y - self.mu) / self.sd).astype(np.float32)
    def inverse_transform(self, yhat: np.ndarray) -> np.ndarray:
        return (yhat * self.sd + self.mu).astype(np.float32)

# Model Definition
**What:** Stacked LSTM with linear head; sequence window length L=90.  
**Why:** Capture temporal dynamics with a lightweight recurrent baseline.  
**Method choices:** Hidden=64, Layers=2, Dropout=0.2; MSE loss; AdamW optimizer; deterministic CuDNN settings.

In [ ]:
# -------------------- datasets / model --------------------
class SeqDS(Dataset):
    def __init__(self, x: np.ndarray, y: np.ndarray):
        self.x = x
        self.y = y
    def __len__(self): return len(self.y)
    def __getitem__(self, idx):
        return torch.from_numpy(self.x[idx]).float(), torch.from_numpy(self.y[idx]).float()

def make_sequences(panel_x: pd.DataFrame, panel_y: pd.Series, L: int) -> Tuple[np.ndarray,np.ndarray]:
    x = panel_x.to_numpy(dtype=np.float32)
    y = panel_y.to_numpy(dtype=np.float32)
    xs, ys = [], []
    for t in range(L, len(panel_x)):
        xs.append(x[t-L:t])
        ys.append(y[t])
    return np.stack(xs), np.array(ys)[:, None]

class StackedLSTM(nn.Module):
    def __init__(self, in_dim: int, hidden: int, layers: int, dropout: float):
        super().__init__()
        self.lstm = nn.LSTM(input_size=in_dim, hidden_size=hidden, num_layers=layers,
                            batch_first=True, dropout=(dropout if layers > 1 else 0.0))
        self.head = nn.Linear(hidden, 1)
    def forward(self, x):
        o, _ = self.lstm(x)
        h = o[:, -1, :]
        y = self.head(h)
        return y

# Training
**What:** Train on Train/Val with early stopping; **monthly refit** over Test (refit on each month’s first trading day using history `< t`).  
**Why:** Avoid look-ahead.  
**Method choices:** Train-only fit for scalers; deterministic DataLoaders (shuffle=False); inverse-transform **y** before metrics/emit.

In [ ]:
# -------------------- core metrics helpers --------------------
def rmse(a, b): return float(np.sqrt(np.mean((a-b)**2)))
def mae(a, b):  return float(np.mean(np.abs(a-b)))

def theil_u2(y_true: np.ndarray, y_hat: np.ndarray) -> float:
    y = y_true
    ylag = np.r_[np.nan, y[:-1]]
    num = np.nansum((y_hat - y)**2)
    den = np.nansum((y - ylag)**2)
    if den <= 0: return float("nan")
    return float(np.sqrt(num/den))

def pure_sign_returns(y_true: np.ndarray, y_hat: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    # Percentage returns from level prices
    ylag = np.r_[np.nan, y_true[:-1]]
    r_true = (y_true - ylag) / ylag
    r_hat  = (y_hat  - ylag) / ylag
    r_true[~np.isfinite(r_true)] = 0.0
    r_hat[~np.isfinite(r_hat)] = 0.0
    pos = np.sign(r_hat)  # position_t = sign(ŷ_{t+1} − y_t)
    return r_true, r_hat, pos

def dir_accuracy_eps(y_true: np.ndarray, y_hat: np.ndarray, eps: float) -> Tuple[float, float, int]:
    r_true, r_hat, _ = pure_sign_returns(y_true, y_hat)
    true_class = np.sign(r_true)
    pred_class = np.sign(r_hat)
    true_class[np.abs(r_true) < eps] = 0
    pred_class[np.abs(r_hat)  < eps] = 0
    mask = (np.abs(r_true) >= eps)  # coverage is fraction of non-neutral actuals
    if mask.sum() == 0:
        return float("nan"), 0.0, 0
    da = float(np.mean(pred_class[mask] == true_class[mask]))
    cov = float(np.mean(mask))
    n = int(mask.sum())
    return da, cov, n

def sharpe_maxdd_turnover(y_true: np.ndarray, y_hat: np.ndarray, bps: float=0.0) -> Tuple[float,float,int]:
    r_true, r_hat, pos = pure_sign_returns(y_true, y_hat)
    fee = bps/10000.0
    changes = np.abs(np.diff(np.r_[0.0, pos]))  # 1 when position changes, else 0
    turnover = int(np.sum(changes > 0))
    strategy_ret = pos * r_true - fee * changes
    mu = float(np.mean(strategy_ret))
    sd = float(np.std(strategy_ret, ddof=0))
    if sd > 0:
        sharpe = float(np.sqrt(252.0) * mu / sd)
    else:
        sharpe = float("nan")
    curve = np.cumprod(1.0 + strategy_ret)
    if curve.size:
        peak = np.maximum.accumulate(curve)
        dd = 1.0 - curve/peak
        maxdd = float(np.max(dd))
    else:
        maxdd = float("nan")
    # Internal sanity prints
    total_cost_calc = fee * np.sum(changes)
    print(f"[Trading sanity] bps={bps:.1f} mu={mu:.6f} sd={sd:.6f} n_changes={turnover} total_cost={total_cost_calc:.6f}")
    return sharpe, maxdd, turnover

# -------------------- training utils --------------------
def train_loop(model, dl_tr, dl_va, epochs, patience, device):
    opt = torch.optim.AdamW(model.parameters(), lr=LR)
    crit = nn.MSELoss()
    best = math.inf
    wait = 0
    hist_tr, hist_va = [], []
    best_state = None
    for ep in range(1, epochs+1):
        model.train()
        tr_loss = 0.0
        for bx, by in dl_tr:
            bx = bx.to(device); by = by.to(device)
            opt.zero_grad()
            pred = model(bx)
            loss = crit(pred, by)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            tr_loss += loss.item() * len(bx)
        tr_loss /= max(1, len(dl_tr.dataset))
        model.eval()
        with torch.no_grad():
            va_loss = 0.0
            for bx, by in dl_va:
                bx = bx.to(device); by = by.to(device)
                pred = model(bx)
                loss = crit(pred, by)
                va_loss += loss.item() * len(bx)
            va_loss /= max(1, len(dl_va.dataset))
        hist_tr.append(tr_loss); hist_va.append(va_loss)
        if va_loss < best - 1e-9:
            best = va_loss
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                break
    if best_state is not None:
        model.load_state_dict(best_state)
    return hist_tr, hist_va

def predict_one_step(model, x_seq: np.ndarray) -> float:
    with torch.no_grad():
        t = torch.from_numpy(x_seq[None, ...]).float().to(DEVICE)
        y = model(t).cpu().numpy().ravel()[0]
    return float(y)

# -------------------- month refit boundaries --------------------
def first_trading_days(df: pd.DataFrame) -> List[pd.Timestamp]:
    te = df[(df["date"]>=TEST_START) & (df["date"]<=TEST_END)].copy()
    te["ym"] = te["date"].dt.to_period("M")
    return te.groupby("ym")["date"].min().tolist()

# -------------------- plotting --------------------
def save_training_curves(path: Path, tr: List[float], va: List[float]):
    fig = plt.figure(figsize=(6,4))
    plt.plot(tr, label="train")
    plt.plot(va, label="val")
    plt.xlabel("epoch"); plt.ylabel("MSE (y on training scale)")
    plt.legend(); plt.tight_layout()
    fig.savefig(path, dpi=160)
    plt.close(fig)

# Evaluation
**What:** Zero-preservation audit, inverse-transform to level, compute metrics and trading diagnostics on Test.  
**Why:** Evidence that scaling policy preserves zeros; report comparable metrics on level prices; DAε uses ε=0.0010; trading metrics use pure-sign rule.  
**Method choices:** Coverage reported as fraction of non-neutral actuals; turnover counts position changes; Sharpe annualised with √252.

In [ ]:
# -------------------- zero-preservation check --------------------
def emit_zero_preservation_check(raw_df: pd.DataFrame,
                                 scaled_df: pd.DataFrame,
                                 cols: List[str],
                                 out_csv: Path,
                                 atol: float = 1e-12):
    rows = []
    total_violations = 0
    if not cols:
        pd.DataFrame([{"column":"<none>", "raw_zero_count":0, "preserved_zero_count":0,
                       "violations":0, "ok": True, "zeros_after_total":0}]).to_csv(out_csv, index=False)
        return
    for c in cols:
        raw = raw_df[c].to_numpy()
        sca = scaled_df[c].to_numpy()
        mask_raw_zero = np.isfinite(raw) & (raw == 0)
        n_raw_zeros = int(mask_raw_zero.sum())
        n_preserved = int(np.isclose(sca[mask_raw_zero], 0.0, atol=atol, rtol=0).sum())
        violations = max(int(n_raw_zeros - n_preserved), 0)
        total_violations += violations
        rows.append({
            "column": c,
            "raw_zero_count": n_raw_zeros,
            "preserved_zero_count": n_preserved,
            "violations": violations,
            "ok": (violations == 0),
            "zeros_after_total": int(np.isclose(sca, 0.0, atol=atol, rtol=0).sum())
        })
    pd.DataFrame(rows).to_csv(out_csv, index=False)
    if total_violations > 0:
        raise RuntimeError(f"Zero-preservation check failed: {total_violations} violations")

# Outputs and Artefacts
**What:** Persist predictions (level), metrics, run_config, training curves, zero-preservation checks, manifests, and final ZIP.  
**Why:** Meet finalisation checklist and enable reproducibility.  
**Method choices:** Per-ticker directories; features parity hash; assert required files and row counts; build bundle with SHA256.

In [ ]:
# -------------------- provenance / manifests --------------------
def write_env_manifest(run_root: Path):
    lines = [f"python: {platform.python_version()}"]
    try:
        import numpy, pandas, torch as _torch
        lines.append(f"numpy: {numpy.__version__}")
        lines.append(f"pandas: {pd.__version__}")
        lines.append(f"torch: {_torch.__version__}")
        try:
            lines.append(f"cuda: {_torch.version.cuda}")
            lines.append(f"cudnn: {_torch.backends.cudnn.version()}")
        except Exception:
            pass
    except Exception:
        pass
    (run_root / "env_manifest.txt").write_text("\n".join(lines) + "\n", encoding="utf-8")

def write_file_hashes(run_root: Path):
    rows = []
    for r, _, fs in os.walk(run_root):
        for fn in fs:
            p = Path(r) / fn
            h = hashlib.sha256()
            with open(p, "rb") as f:
                for chunk in iter(lambda: f.read(1<<20), b""):
                    h.update(chunk)
            rows.append({"path": str(p.relative_to(run_root)).replace("\\","/"),
                         "size": p.stat().st_size,
                         "sha256": h.hexdigest()})
    (run_root / "file_hashes.json").write_text(json.dumps(rows, indent=2), encoding="utf-8")

def write_provenance(run_root: Path):
    rows = [{
        "repo_name": "keras-team/keras-io",
        "repo_url": "https://github.com/keras-team/keras-io",
        "licence": "Apache-2.0",
        "commit_sha": COMMIT_SHA,
        "imported_files": "examples/timeseries/timeseries_weather_forecasting.py",
        "adaptation_notes": "Adapted stacked-LSTM pattern; project-specific loaders, zero-preserving scaler, monthly refit, metrics."
    }]
    (run_root / "code_provenance.csv").write_text(pd.DataFrame(rows).to_csv(index=False), encoding="utf-8")
    tpd = run_root / "third_party_licenses"
    tpd.mkdir(parents=True, exist_ok=True)
    (tpd / "README.txt").write_text(f"Keras-io example adapted\nCommit: {COMMIT_SHA}\nLicence: Apache-2.0\n", encoding="utf-8")

# -------------------- finalisation / packaging --------------------
def features_sha256(feats: List[str]) -> str:
    blob = "\n".join([str(x) for x in feats]).encode("utf-8")
    return hashlib.sha256(blob).hexdigest()

def write_outputs_index(run_root: Path):
    idx = {}
    for t in TICKERS:
        d = run_root / MODEL_ID / t
        idx[t] = sorted([p.name for p in d.glob("*")]) if d.exists() else []
    (run_root / "outputs_index.json").write_text(json.dumps(idx, indent=2), encoding="utf-8")

def assert_checklist_files(run_root: Path):
    root = run_root / MODEL_ID
    required_top = [
        run_root / "features_manifest.json",
        run_root / "env_manifest.txt",
        run_root / "file_hashes.json",
        run_root / "code_provenance.csv",
        run_root / "outputs_index.json",
        run_root / "third_party_licenses" / "README.txt",
        run_root / "README.txt",
    ]
    for p in required_top:
        if not p.exists():
            raise RuntimeError(f"Missing top-level artefact: {p}")
    for t in TICKERS:
        d = root / t
        if not d.exists():
            raise RuntimeError(f"Missing ticker directory: {d}")
        req = {
            "predictions": d / f"predictions_{MODEL_ID}_{t}.csv",
            "metrics":     d / f"metrics_{MODEL_ID}_{t}.json",
            "run_config":  d / f"run_config_{MODEL_ID}_{t}.json",
            "curves":      d / f"training_curves_{MODEL_ID}_{t}.png",
            "zero_check":  d / f"zero_preservation_check_{t}.csv",
        }
        for name, p in req.items():
            if not p.exists():
                raise RuntimeError(f"[{t}] missing {name}: {p}")
        n_pred = sum(1 for _ in open(req["predictions"], "r", encoding="utf-8")) - 1
        if n_pred != ASSERT_N_TEST:
            raise RuntimeError(f"[{t}] predictions rows={n_pred}, expected {ASSERT_N_TEST}")
        m = json.loads(req["metrics"].read_text(encoding="utf-8"))
        if m.get("n", None) != ASSERT_N_TEST:
            raise RuntimeError(f"[{t}] metrics 'n' must be {ASSERT_N_TEST}, got {m.get('n')}")

def write_readme(run_root: Path, feats_used: List[str]):
    txt = (
        "LSTM_SE pack\n"
        "============\n\n"
        f"Model ID: {MODEL_ID}\n"
        f"Tickers: {', '.join(TICKERS)}\n"
        "Splits:\n"
        f"  - Train: {TRAIN_START} to {TRAIN_END}\n"
        f"  - Val:   {VAL_START} to {VAL_END}\n"
        f"  - Test:  {TEST_START} to {TEST_END} (n=146)\n\n"
        "Contents\n"
        "--------\n"
        "Top-level:\n"
        "  - features_manifest.json\n"
        "  - env_manifest.txt\n"
        "  - file_hashes.json\n"
        "  - code_provenance.csv\n"
        "  - outputs_index.json\n"
        "  - third_party_licenses/\n"
        "  - README.txt\n\n"
        "Per ticker under LSTM_SE/<TICKER>/:\n"
        f"  - predictions_{MODEL_ID}_<TICKER>.csv\n"
        f"  - metrics_{MODEL_ID}_<TICKER>.json\n"
        f"  - run_config_{MODEL_ID}_<TICKER>.json\n"
        f"  - training_curves_{MODEL_ID}_<TICKER>.png\n"
        "  - zero_preservation_check_<TICKER>.csv\n\n"
        "Reproduction\n"
        "------------\n"
        "1) Ensure final_inputs/*.csv exist with the expected schema.\n"
        "2) Run this script from the project root.\n"
        "3) The finalised ZIP is created automatically in the project root.\n\n"
        "Evidence (global invariants)\n"
        "----------------------------\n"
        f"- Clock: {json.dumps(CLOCK_INVARIANTS)}\n"
        f"- Target provenance: {json.dumps(TARGET_PROVENANCE)}\n"
        f"- Features parity hash (LSTM_SE): {features_sha256(feats_used)}\n"
        f"- Target scaling: {'standard' if SCALE_TARGET else 'none'}; inverse-transform applied before writing predictions and computing metrics.\n"
        "- Errors (RMSE/MAE/U2) are computed on level prices; Directional Accuracy uses daily returns with epsilon = 0.0010.\n"
        "- Trading diagnostics use the pure-sign rule; costs apply only when the position changes; Turnover counts position changes.\n"
        "- Cross-model features parity: compare this SHA256 across LSTM_SE, TRANSFORMER, and HYBRID packs.\n"
    )
    (run_root / "README.txt").write_text(txt, encoding="utf-8")

def build_zip(run_root: Path, outfile: Optional[Path] = None) -> Path:
    run_root = run_root.resolve()
    if outfile is None:
        stamp = time.strftime("%Y%m%d_%H%M%S", time.gmtime())
        outfile = run_root.parent / f"{run_root.name}_pack_{stamp}.zip"
    with zipfile.ZipFile(outfile, "w", compression=zipfile.ZIP_DEFLATED, allowZip64=True) as zf:
        for p in sorted(run_root.rglob("*")):
            if p.is_dir(): continue
            arc = str(p.relative_to(run_root)).replace("\\","/")
            zf.write(p, arcname=arc)
    h = hashlib.sha256(outfile.read_bytes()).hexdigest()
    (run_root / "bundle_sha256.txt").write_text(f"{outfile.name}  {h}\n", encoding="utf-8")
    print("Pack created:", outfile)
    print("SHA256:", (run_root / "bundle_sha256.txt").read_text(encoding="utf-8").strip())
    return outfile

def finalise_pack(feats_used: List[str]):
    write_outputs_index(OUT_ROOT)
    write_readme(OUT_ROOT, feats_used)
    assert_checklist_files(OUT_ROOT)
    build_zip(OUT_ROOT)

# Per-ticker Runner
**What:** Execute monthly refit forecasting for one ticker and write artefacts.  
**Why:** Deterministic, leakage-safe forecasts and complete outputs per ticker.  
**Method choices:** Refit on first trading day per month; use history `< t`; scalers re-fit per refit using Train-only policy.

In [ ]:
# -------------------- core runner (per ticker) --------------------
def run_one_ticker(ticker: str):
    out_dir = MODEL_DIR / ticker
    out_dir.mkdir(parents=True, exist_ok=True)

    df = load_frame(ticker)
    splits = fixed_splits(df)
    feats = infer_features(df)
    sent_cols = sentiment_cols_from(feats)
    non_sent_cols = [c for c in feats if c not in sent_cols]

    tr_raw = splits["train"].copy()
    va_raw = splits["val"].copy()
    te_raw = splits["test"].copy()
    y_col = "Target"

    zs_sent = ZeroPreservingStandardScaler(sent_cols).fit(tr_raw) if sent_cols else None
    zs_other = StdScaler(non_sent_cols).fit(tr_raw) if non_sent_cols else None

    def apply_feat_scalers(df_in: pd.DataFrame) -> pd.DataFrame:
        z = df_in.copy()
        if zs_sent:  z = zs_sent.transform(z)
        if zs_other: z = zs_other.transform(z)
        return z

    tr = apply_feat_scalers(tr_raw)
    va = apply_feat_scalers(va_raw)
    te = apply_feat_scalers(te_raw)

    emit_zero_preservation_check(
        pd.concat([tr_raw, va_raw, te_raw], ignore_index=True),
        pd.concat([tr, va, te],    ignore_index=True),
        sent_cols,
        out_dir / f"zero_preservation_check_{ticker}.csv"
    )

    y_scaler_cls = StdScaler1D if SCALE_TARGET else Identity1D
    y_scaler = y_scaler_cls().fit(tr_raw[y_col].to_numpy())

    y_tr_scaled = y_scaler.transform(tr_raw[y_col].to_numpy())
    y_va_scaled = y_scaler.transform(va_raw[y_col].to_numpy())
    x_tr, y_tr = make_sequences(tr[feats], pd.Series(y_tr_scaled, index=tr.index), L)
    x_va, y_va = make_sequences(va[feats], pd.Series(y_va_scaled, index=va.index), L)

    dl_tr = DataLoader(SeqDS(x_tr, y_tr), batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    dl_va = DataLoader(SeqDS(x_va, y_va), batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    model0 = StackedLSTM(in_dim=len(feats), hidden=HIDDEN, layers=LAYERS, dropout=DROPOUT).to(DEVICE)
    tr_hist, va_hist = train_loop(model0, dl_tr, dl_va, EPOCHS, EARLY_STOP_PATIENCE, DEVICE)
    save_training_curves(out_dir / f"training_curves_{MODEL_ID}_{ticker}.png", tr_hist, va_hist)

    test_dates = te_raw["date"].tolist()
    first_days = first_trading_days(df)
    idx_test = te_raw.reset_index(drop=True)
    i_map = {d:i for i,d in enumerate(idx_test["date"])}

    # Monthly refit: first trading day per month, exclude day t from history
    refit_indices = sorted(set([0] + [i_map[d] for d in first_days if d in i_map]))
    chunks = []
    for k, start in enumerate(refit_indices):
        end = (refit_indices[k+1]-1) if (k+1)<len(refit_indices) else (len(idx_test)-1)
        chunks.append((start, end))

    y_true_all, y_hat_all, in_flag_all = [], [], []
    refit_dates_log, epochs_per_refit = [], []
    parameter_count = int(sum(p.numel() for p in model0.parameters()))

    for (start_i, end_i) in chunks:
        d0 = te_raw.iloc[start_i]["date"]
        refit_dates_log.append(str(pd.to_datetime(d0).date()))

        sub_hist = df[df["date"] < d0].copy()
        tr_sub = sub_hist[(sub_hist["date"]>=TRAIN_START) & (sub_hist["date"]<=TRAIN_END)].copy()
        va_sub = sub_hist[(sub_hist["date"]>=VAL_START) & (sub_hist["date"]<=VAL_END)].copy()

        zs_sent_m = ZeroPreservingStandardScaler(sent_cols).fit(tr_sub) if sent_cols else None
        zs_other_m = StdScaler(non_sent_cols).fit(tr_sub) if non_sent_cols else None
        y_scaler_m = (StdScaler1D() if SCALE_TARGET else Identity1D()).fit(tr_sub[y_col].to_numpy())

        def apply_scalers_now(df_in: pd.DataFrame) -> pd.DataFrame:
            z = df_in.copy()
            if zs_sent_m:  z = zs_sent_m.transform(z)
            if zs_other_m: z = zs_other_m.transform(z)
            return z

        tr_s = apply_scalers_now(tr_sub)
        va_s = apply_scalers_now(va_sub)

        y_tr_s = y_scaler_m.transform(tr_sub[y_col].to_numpy())
        y_va_s = y_scaler_m.transform(va_sub[y_col].to_numpy())

        x_tr_m, y_tr_m = make_sequences(tr_s[feats], pd.Series(y_tr_s, index=tr_s.index), L)
        x_va_m, y_va_m = make_sequences(va_s[feats], pd.Series(y_va_s, index=va_s.index), L)

        dl_tr_m = DataLoader(SeqDS(x_tr_m, y_tr_m), batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        dl_va_m = DataLoader(SeqDS(x_va_m, y_va_m), batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

        model_m = StackedLSTM(in_dim=len(feats), hidden=HIDDEN, layers=LAYERS, dropout=DROPOUT).to(DEVICE)
        tr_hist_m, va_hist_m = train_loop(model_m, dl_tr_m, dl_va_m, EPOCHS, EARLY_STOP_PATIENCE, DEVICE)
        epochs_per_refit.append(len(tr_hist_m))
        parameter_count = int(sum(p.numel() for p in model_m.parameters()))

        for i in range(start_i, end_i+1):
            ctx_raw = pd.concat([tr_sub, va_sub, te_raw.iloc[:i]], ignore_index=True)
            ctx_s   = apply_scalers_now(ctx_raw)
            x_seq = ctx_s[feats].to_numpy(dtype=np.float32)[-L:]
            yhat_scaled = predict_one_step(model_m, x_seq)
            yhat_level  = float(y_scaler_m.inverse_transform(np.array([yhat_scaled], dtype=np.float32))[0])
            y_true_all.append(float(te_raw.iloc[i][y_col]))
            y_hat_all.append(yhat_level)
            in_flag_all.append(0)

    # -------------------- metrics and emissions --------------------
    y_true = np.array(y_true_all, dtype=float)
    y_hat  = np.array(y_hat_all,  dtype=float)
    assert len(y_true) == ASSERT_N_TEST == len(y_hat)

    # Level-scale assertion and sanity ratio
    mean_true, mean_hat = float(np.nanmean(y_true)), float(np.nanmean(y_hat))
    if not (0.25 <= (mean_hat + 1e-9) / (mean_true + 1e-9) <= 4.0):
        raise RuntimeError(
            f"[{ticker}] Scale sanity check failed: mean(y_hat)={mean_hat:.2f}, mean(y_true)={mean_true:.2f}. "
            "This indicates an inverse-transform or target mapping problem."
        )

    # Metrics on level scale
    rmse_v = rmse(y_true, y_hat)
    mae_v  = mae(y_true, y_hat)
    u2_v   = theil_u2(y_true, y_hat)

    # Directional Accuracy — epsilon-gated, coverage on non-neutral actuals
    da_v, cov_v, n_da = dir_accuracy_eps(y_true, y_hat, EPSILON_RET)

    # Trading metrics — pure-sign rule, costs only on position changes
    sh0, dd0, turn = sharpe_maxdd_turnover(y_true, y_hat, bps=0.0)
    sh10, dd10, _  = sharpe_maxdd_turnover(y_true, y_hat, bps=10.0)

    # Predictions file (level scale)
    pred_df = pd.DataFrame({
        "date": pd.to_datetime(te_raw["date"]).iloc[:len(y_true)].to_list(),
        "y_true": y_true,
        "y_hat": y_hat,
        "residual": y_true - y_hat,
        "in_sample_flag": in_flag_all
    })
    out_dir.mkdir(parents=True, exist_ok=True)
    pred_df.to_csv(out_dir / f"predictions_{MODEL_ID}_{ticker}.csv", index=False)

    # Metrics JSON (include Turnover and n)
    metrics = {
        "RMSE": rmse_v, "MAE": mae_v, "U2": u2_v,
        "DA_epsilon": da_v, "Coverage": cov_v,
        "n": ASSERT_N_TEST,
        "n_da": n_da,
        "Sharpe_0bps": sh0, "Sharpe_10bps": sh10,
        "MaxDD_0bps": dd0, "MaxDD_10bps": dd10,
        "Turnover": int(turn)
    }
    (out_dir / f"metrics_{MODEL_ID}_{ticker}.json").write_text(json.dumps(metrics, indent=2), encoding="utf-8")

    # Run config JSON (architecture, cadence, parity)
    run_cfg = {
        "model_id": MODEL_ID,
        "ticker": ticker,
        "splits": {"train": [TRAIN_START, TRAIN_END], "val": [VAL_START, VAL_END], "test": [TEST_START, TEST_END], "n_test": ASSERT_N_TEST},
        "cadence": "monthly_refit",
        "refit_dates": refit_dates_log,
        "seeds": {"python": RANDOM_SEED, "numpy": RANDOM_SEED, "torch": RANDOM_SEED, "cuda_seed": RANDOM_SEED if torch.cuda.is_available() else None},
        "features_used": feats,
        "features_sha256": features_sha256(feats),
        "sentiment_zero_columns": [c for c in feats if c in sent_cols],
        "parameter_count": int(parameter_count),
        "window_length": L,
        "batch_size": BATCH_SIZE,
        "epochs_per_refit": epochs_per_refit,
        "hyperparameters": {"hidden": HIDDEN, "layers": LAYERS, "dropout": DROPOUT, "lr": LR, "loss": "MSE", "optimiser": "AdamW", "early_stop_patience": EARLY_STOP_PATIENCE},
        "target_scaling": "standard" if SCALE_TARGET else "none",
        "inverse_transform_applied": True,
        "clock_invariants": CLOCK_INVARIANTS,
        "target_provenance": TARGET_PROVENANCE,
        "provenance": {"repos": [{"name": "keras-io example", "commit_sha": COMMIT_SHA, "licence": "Apache-2.0"}]},
        "device": DEVICE
    }
    (out_dir / f"run_config_{MODEL_ID}_{ticker}.json").write_text(json.dumps(run_cfg, indent=2), encoding="utf-8")

    return feats

# Features Manifest
**What:** Record features used and parity hash across tickers.  
**Why:** Evidence identical feature set across files and models.  
**Method choices:** SHA256 of ordered feature list; list sentiment-zero columns for audits.

In [ ]:
# -------------------- feature manifest --------------------
def write_features_manifest(all_feats_lists: List[List[str]]):
    base = all_feats_lists[0]
    ok = all(base == f for f in all_feats_lists[1:])
    if not ok:
        raise RuntimeError("Features parity mismatch across tickers")
    features_manifest = {
        "features_used": base,
        "features_sha256": features_sha256(base),
        "sentiment_zero_columns": [c for c in base if c.startswith(("Tw_","Rd_","Nw_SP500_"))],
        "intended_cross_model_parity": "Compare this SHA256 across packs to evidence parity."
    }
    (OUT_ROOT / "features_manifest.json").write_text(json.dumps(features_manifest, indent=2), encoding="utf-8")
    (OUT_ROOT / "cross_model_features_parity_stub.json").write_text(
        json.dumps({"model_id": MODEL_ID, "features_sha256": features_sha256(base)}, indent=2), encoding="utf-8"
    )
    return base

# Driver
**What:** Run all tickers, write manifests, and finalise bundle.  
**Why:** Produce a complete, portable artefact pack.  
**Method choices:** Deterministic order over tickers; asserts required outputs; builds ZIP and writes bundle hash.


In [ ]:
# -------------------- main --------------------
def main():
    feats_lists = []
    for t in TICKERS:
        feats_lists.append(run_one_ticker(t))
    feats_used = write_features_manifest(feats_lists)
    write_provenance(OUT_ROOT)
    write_env_manifest(OUT_ROOT)
    write_file_hashes(OUT_ROOT)
    finalise_pack(feats_used)

if __name__ == "__main__":
    main()

In [ ]:
# Build (if missing) and download the zip from Colab to your PC
from pathlib import Path
import zipfile, os

zip_path = Path("/content/lstm_se.zip")
root = Path("/content/LSTM_SE_FINAL")  # where the pipeline writes outputs

if not root.exists():
    raise FileNotFoundError("LSTM_SE_FINAL not found. Run the training/build cell first.")

# Create the zip if it isn't there already
if not zip_path.exists():
    with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED, allowZip64=True) as zf:
        for p in sorted(root.rglob("*")):
            if p.is_file():
                arc = "lstm_se/" + str(p.relative_to(root)).replace("\\","/")
                zf.write(str(p), arcname=arc)
print("Zip ready at:", zip_path, "size:", zip_path.stat().st_size, "bytes")

from google.colab import files
files.download(str(zip_path))  # triggers browser download